In [1]:
import numpy as np

class Board:
    def __init__(self):
        self.board=np.full(shape=(8,8), fill_value="e")
        self.board[4][4] = "w"
        self.board[4][3] = "b"
        self.board[3][3] = "w"
        self.board[3][4] = "b"

GAME_STATE = True
EMPTY = "e"
WHITE = "w"
BLACK = "b"
DIRECTIONS = ((0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1))

def opposite_color(color):
    if color == BLACK:
        return WHITE
    else:
        return BLACK

def is_valid_move(board, player, row, col):
    if board[row][col] != 'e':
        return False

    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            r, c = row + i, col + j
            if r < 0 or r >= 8 or c < 0 or c >= 8:
                continue
            if board[r][c] == opposite_color(player):
                while True:
                    r, c = r + i, c + j
                    if r < 0 or r >= 8 or c < 0 or c >= 8:
                        break
                    if board[r][c] == player:
                        return True
                    if board[r][c] == 'e':
                        break

    return False

def get_valid_moves(board, player):
    valid_moves = []
    for row in range(8):
        for col in range(8):
            if is_valid_move(board, player, row, col):
                valid_moves.append((row, col))
    return valid_moves

def flip_pieces(board, player, row, col):
    for direction in DIRECTIONS:
        r, c = row + direction[0], col + direction[1]
        if r < 0 or r >= 8 or c < 0 or c >= 8 or board[r][c] == player or board[r][c] == EMPTY:
            continue
        pieces_to_flip = []
        while board[r][c] != player:
            pieces_to_flip.append((r, c))
            r += direction[0]
            c += direction[1]
            if r < 0 or r >= 8 or c < 0 or c >= 8 or board[r][c] == EMPTY:
                break
        else:
            for piece in pieces_to_flip:
                board[piece[0]][piece[1]] = player
    board[row][col] = player

def get_winner(board):
    white_count = np.count_nonzero(board == WHITE)
    black_count = np.count_nonzero(board == BLACK)
    if white_count > black_count:
        return WHITE
    elif black_count > white_count:
        return BLACK
    else:
        return EMPTY

def make_move(board, player, row, col):
    if row < 0 or row >= len(board) or col < 0 or col >= len(board[0]):
        return False
    if not is_valid_move(board, player, row, col):
        return False
    flip_pieces(board, player, row, col)
    return True

def get_score(board):
    black_count = 0
    white_count = 0
    
    for row in range(8):
        for col in range(8):
            if board[row][col] == 'b':
                black_count += 1
            elif board[row][col] == 'w':
                white_count += 1
    
    return (black_count, white_count)



In [2]:
import pygame
import sys


WINDOW_SIZE = [450, 500]
# Define some colors
BLACK_COLOR = (0, 0, 0)
WHITE_COLOR = (255, 255, 255)
GREEN_COLOR = (0, 255, 0)
RED_COLOR = (255, 0, 0)

SQUARE_SIZE = 56.25

def draw_lines():
    # Draw the horizontal grid lines
    for i in range(8):
        start_pos = (0, i * SQUARE_SIZE + SQUARE_SIZE )
        end_pos = (450, i * SQUARE_SIZE + SQUARE_SIZE)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

    # Draw the vertical grid lines
    for i in range(8):
        start_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 0)
        end_pos = (i * SQUARE_SIZE + SQUARE_SIZE, 450)
        pygame.draw.line(screen, BLACK_COLOR, start_pos, end_pos)

def is_game_over(board, current_player):
    black_moves = get_valid_moves(board, BLACK)
    white_moves = get_valid_moves(board, WHITE)
    if current_player == "b" and len(black_moves) == 0:
        return True
    elif current_player == "w" and len(white_moves) == 0:
        return True
    else:
        return False

def show_winner(screen, winner):
    font = pygame.font.Font(None, 50)
    text = font.render(f"Winner: {winner}", True, RED_COLOR)
    text_rect = text.get_rect(center=(WINDOW_SIZE[0] // 2, 50))
    screen.blit(text, text_rect)

def show_menu(screen):
    # Fill the background with white color
    screen.fill(WHITE_COLOR)

    # Draw the title
    font = pygame.font.Font(None, 48)
    text = font.render("Main Menu", True, BLACK_COLOR)
    text_rect = text.get_rect(center=(WINDOW_SIZE[0] // 2, 100))
    screen.blit(text, text_rect)

    # Define the button positions and sizes
    button_width = 200
    button_height = 50
    button_x = WINDOW_SIZE[0] // 2 - button_width // 2
    button_y = 200

    # Create a list of button rects
    button_rects = []
    for i, button_text in enumerate(["Start Game", "Quit"]):
        rect = pygame.Rect(button_x, button_y + i * (button_height + 10), button_width, button_height)
        button_rects.append(rect)

        # Draw the button
        pygame.draw.rect(screen, BLACK_COLOR, rect, 2)
        font = pygame.font.Font(None, 36)
        text = font.render(button_text, True, BLACK_COLOR)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)

    # Update the display
    pygame.display.flip()

    return button_rects

def render_score(surface, font, score):
    if current_player == "b":
        player = "Black"
    else:
        player = "White"
    black_score, white_score = score
    text = f"Black score: {black_score}   White score: {white_score} Now: {player}"
    text_surface = font.render(text, True, (0, 0, 0))
    surface.blit(text_surface, (10, 450))

def draw_board(screen, board):
    for row in range(8):
        for col in range(8):
            x = col * SQUARE_SIZE
            y = row * SQUARE_SIZE
            if board[row][col] == BLACK:
                pygame.draw.circle(screen, BLACK_COLOR, (x+SQUARE_SIZE//2, y+SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)
            elif board[row][col] == WHITE:
                pygame.draw.circle(screen, WHITE_COLOR, (x+SQUARE_SIZE//2, y+SQUARE_SIZE//2), SQUARE_SIZE//2 - 5)

# This sets the WIDTH and HEIGHT of each grid location
WIDTH = 50
HEIGHT = 50

# This sets the margin between each cell
MARGIN = 5

# Initialize pygame
pygame.init()

# Set the size of the screen

screen = pygame.display.set_mode(WINDOW_SIZE)
font = pygame.font.SysFont("Arial", 24)

# Set the title of the window
pygame.display.set_caption("Reversi Game")

# Loop until the user clicks the close button
done = False

# Used to manage how fast the screen updates
clock = pygame.time.Clock()
board = 0
# Create the board
# board = Board()

# Set the current player to black
current_player = BLACK
# -------- Main Program Loop -----------
while not done:

     # --- Show menu ---
    option = show_menu(screen)

    # --- Start new game ---
    if option == '1':
        board = Board()
        current_player = 'b'
        winner = None
        # --- Game logic should go here

        # --- Drawing code should go here
        # Clear the screen
        screen.fill(GREEN_COLOR)
        draw_board(screen, board.board)
        if is_game_over(board.board, current_player) == True:
            black_score, white_score = get_score(board.board)
            if black_score > white_score:
                winner = "BLACK"
            elif black_score < white_score:
                winner = "WHITE"
            else:
                winner = "DRAW"
            show_winner(screen, winner)
        draw_lines()
        render_score(screen, font, get_score(board.board))

    # --- Quit ---
    elif option == '2':
        done = True

    # --- Main game loop ---
    else:
        # --- Game logic and drawing code goes here ---
    # --- Main event loop
         for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
            elif event.type == pygame.MOUSEBUTTONDOWN:
                # Get the position of the mouse
                pos = pygame.mouse.get_pos()
                # Convert the position into board coordinates
                column = pos[0] // (WIDTH + MARGIN)
                row = pos[1] // (HEIGHT + MARGIN)
                # Make a move if it is valid
                if make_move(board.board, current_player, row, column):
                    # Switch the player
                    current_player = opposite_color(current_player)

        # Draw the board


        # Update the screen
            pygame.display.flip()

        # Limit the frame rate
            clock.tick(60)

# Close the window and quit.
pygame.quit()

pygame 2.3.0 (SDL 2.24.2, Python 3.11.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'board' is not defined

: 